In [ ]:
import os
import sys

sys.path.append('../..')
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from ase.build import make_supercell
from ase import Atoms
from ase.optimize import BFGS, LBFGS
from HybridFF import HybridFF
from Utilities import show_results, write_xyz, validate
from Constants import EV_TO_KJ, H_TO_KJ, BOHR_TO_ANGSTROM
from Barostat import MonteCarloBarostat

In [ ]:
DATA = np.load('../../../data/test_sets/DATA_CSP_ranking.npy', allow_pickle=True).item()

In [ ]:
hybrid_ff = HybridFF(pol0=False) 
barostat = MonteCarloBarostat(T=0, P=100000, ds=1e-3, aniso=True)

In [ ]:
SYSTEM = 'XXVI'
RESULTS = {}
RESULTS[SYSTEM] = {}
MIN_CELLS = {}
MIN_CELLS = np.load(f'data/MIN_CELLS_{SYSTEM}.npy', allow_pickle=True).item()

In [ ]:
for poly_key in DATA[SYSTEM]:
    if poly_key not in MIN_CELLS:
        try:
            RESULTS[SYSTEM][poly_key] = {}  
            MIN_CELLS[poly_key] = {}
            print(poly_key, DATA[SYSTEM][poly_key]['PBE+MBD'])
            mol_size = len(DATA[SYSTEM][poly_key]['monomer_symbols'])
            elements_uc = DATA[SYSTEM][poly_key]['uc_symbols']
            coords_uc = DATA[SYSTEM][poly_key]['uc_coordinates'][None]
            cells = DATA[SYSTEM][poly_key]['cell'][None]    
            atoms = hybrid_ff.initialize(elements_uc, coords_uc, cells, mol_size) 
            atoms = barostat.minimize(atoms, n_steps=200)
            MIN_CELLS[poly_key]['cell'] = np.array(atoms.get_cell()).astype(np.float32)[None]
            MIN_CELLS[poly_key]['coords'] = np.array(atoms.get_positions()).astype(np.float32)[None]
            np.save(f'data/MIN_CELLS_{SYSTEM}.npy', MIN_CELLS)
        except:
            continue